# Loan predictions

## Problem Statement

We want to automate the loan eligibility process based on customer details that are provided as online application forms are being filled. You can find the dataset [here](https://drive.google.com/file/d/1h_jl9xqqqHflI5PsuiQd_soNYxzFfjKw/view?usp=sharing). These details concern the customer's Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and other things as well. 

|Variable| Description|
|------------- |-------------|
|Loan_ID| Unique Loan ID|
|Gender| Male/ Female|
|Married| Applicant married (Y/N)|
|Dependents| Number of dependents|
|Education| Applicant Education (Graduate/ Under Graduate)|
|Self_Employed| Self employed (Y/N)|
|ApplicantIncome| Applicant income|
|CoapplicantIncome| Coapplicant income|
|LoanAmount| Loan amount in thousands|
|Loan_Amount_Term| Term of loan in months|
|Credit_History| credit history meets guidelines|
|Property_Area| Urban/ Semi Urban/ Rural|
|Loan_Status| Loan approved (Y/N)



### Explore the problem in following stages:

1. Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome
2. Data Exploration – looking at categorical and continuous feature summaries and making inferences about the data.
3. Data Cleaning – imputing missing values in the data and checking for outliers
4. Feature Engineering – modifying existing variables and creating new ones for analysis
5. Model Building – making predictive models on the data

## 1. Hypothesis Generation

Generating a hypothesis is a major step in the process of analyzing data. This involves understanding the problem and formulating a meaningful hypothesis about what could potentially have a good impact on the outcome. This is done BEFORE looking at the data, and we end up creating a laundry list of the different analyses which we can potentially perform if data is available.

#### Possible hypotheses
Which applicants are more likely to get a loan

1. Applicants having a credit history 
2. Applicants with higher applicant and co-applicant incomes
3. Applicants with higher education level
4. Properties in urban areas with high growth perspectives

Do more brainstorming and create some hypotheses of your own. Remember that the data might not be sufficient to test all of these, but forming these enables a better understanding of the problem.

To avoid biases I will be excluding the applicant gender and martial statuses. The number of dependents and whether there is suffucient total (applicant + coapplicant) income should be sufficient to determine their ability to meet their family financial needs.

## 2. Data Exploration
Let's do some basic data exploration here and come up with some inferences about the data. Go ahead and try to figure out some irregularities and address them in the next section. 

In [505]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px

df = pd.read_csv("../data/data.csv") 
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.00,NaN,360.00,1.00,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,"1,508.00",128.00,360.00,1.00,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.00,66.00,360.00,1.00,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,"2,358.00",120.00,360.00,1.00,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.00,141.00,360.00,1.00,Urban,Y


In [506]:
df.shape

(614, 13)

In [507]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


One of the key challenges in any data set are missing values. Lets start by checking which columns contain missing values.

In [508]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [509]:
NUMERICAL_FEATURES = [
    "ApplicantIncome",
    "CoapplicantIncome",
    "LoanAmount",
    "Loan_Amount_Term"
]
CATEGORICAL_FEATURES = [
    "Dependents",
    "Education",
    "Self_Employed",
    "Credit_History",
    "Property_Area",
]
TARGET = ["Loan_Status"]
FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES

In [510]:
for col in NUMERICAL_FEATURES:
    print(df[col].value_counts(dropna=False))

2500    9
4583    6
6000    6
2600    6
3333    5
       ..
3244    1
4408    1
3917    1
3992    1
7583    1
Name: ApplicantIncome, Length: 505, dtype: int64
0.00        273
2,500.00      5
2,083.00      5
1,666.00      5
2,250.00      3
           ... 
2,791.00      1
1,010.00      1
1,695.00      1
2,598.00      1
240.00        1
Name: CoapplicantIncome, Length: 287, dtype: int64
NaN       22
120.00    20
110.00    17
100.00    15
160.00    12
          ..
240.00     1
214.00     1
59.00      1
166.00     1
253.00     1
Name: LoanAmount, Length: 204, dtype: int64
360.00    512
180.00     44
480.00     15
NaN        14
300.00     13
240.00      4
84.00       4
120.00      3
60.00       2
36.00       2
12.00       1
Name: Loan_Amount_Term, dtype: int64


Look at some basic statistics for numerical variables.

In [511]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.00,614.00,592.00,600.00,564.00
mean,"5,403.46","1,621.25",146.41,342.00,0.84
std,"6,109.04","2,926.25",85.59,65.12,0.36
min,150.00,0.00,9.00,12.00,0.00
25%,"2,877.50",0.00,100.00,360.00,1.00
50%,"3,812.50","1,188.50",128.00,360.00,1.00
75%,"5,795.00","2,297.25",168.00,360.00,1.00
max,"81,000.00","41,667.00",700.00,480.00,1.00


In [512]:
df.select_dtypes("number").mode()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,2500,0.00,120.00,360.00,1.00


In [513]:
df.skew(axis=0, skipna=True, numeric_only=True)

ApplicantIncome      6.54
CoapplicantIncome    7.49
LoanAmount           2.68
Loan_Amount_Term    -2.36
Credit_History      -1.88
dtype: float64

In [514]:
for col in NUMERICAL_FEATURES:
    px.histogram(df, x=col, width=800, height=500).show()

1. How many applicants have a `Credit_History`? (`Credit_History` has value 1 for those who have a credit history and 0 otherwise)
2. Is the `ApplicantIncome` distribution in line with your expectation? Similarly, what about `CoapplicantIncome`?
3. Tip: Can you see a possible skewness in the data by comparing the mean to the median, i.e. the 50% figure of a feature.



Let's discuss nominal (categorical) variable. Look at the number of unique values in each of them.

In [515]:
df.loc[:, CATEGORICAL_FEATURES].nunique(dropna=False)

Dependents        5
Education         2
Self_Employed     3
Credit_History    3
Property_Area     3
dtype: int64

Explore further using the frequency of different categories in each nominal variable. Exclude the ID obvious reasons.

In [516]:
for col in CATEGORICAL_FEATURES:
    px.histogram(df, x=col, width=800, height=500).show()

### Distribution analysis

Study distribution of various variables. Plot the histogram of ApplicantIncome, try different number of bins.



In [517]:
px.histogram(df, "ApplicantIncome", nbins=250).show()


Look at box plots to understand the distributions. 

In [518]:
for col in NUMERICAL_FEATURES:
    px.box(df, x="Dependents", y=col, points="suspectedoutliers").show()

Look at the distribution of income segregated  by `Education`

In [519]:
px.box(df, x="Education", y="ApplicantIncome", points="suspectedoutliers").show()

Look at the histogram and boxplot of LoanAmount

In [520]:
px.box(df, x="Education", y="LoanAmount", points="suspectedoutliers").show()

There might be some extreme values. Both `ApplicantIncome` and `LoanAmount` require some amount of data munging. `LoanAmount` has missing and well as extreme values values, while `ApplicantIncome` has a few extreme values, which demand deeper understanding. 

### Categorical variable analysis

Try to understand categorical variables in more details using `pandas.DataFrame.pivot_table` and some visualizations.

In [521]:
def count_na(x: pd.Series) -> int:
    return sum(x.isna())

def count_zero(x: pd.Series) -> int:
    return sum(x == 0)


In [522]:
with pd.option_context("display.float_format", "{:,.2f}".format, "display.max_columns", 30):
    for col in CATEGORICAL_FEATURES:
        display(
            df.pivot_table(
                index=col,
                dropna=False,
                aggfunc={x: ["count", "median", "mean", np.std, count_na, count_zero, pd.Series.skew] for x in NUMERICAL_FEATURES},
            )
        )

ApplicantIncome                                             \
                     count count_na count_zero     mean   median skew   
Dependents                                                              
0                      345        0          0 4,917.42 3,598.00 6.26   
1                      102        0          0 5,962.27 4,051.50 3.47   
2                      101        0          0 4,926.78 4,006.00 2.14   
3+                      51        0          0 8,581.22 4,691.00 4.06   

                     CoapplicantIncome                                        \
                 std             count count_na count_zero     mean   median   
Dependents                                                                     
0           5,029.42               345        0        157 1,623.70 1,330.00   
1           5,587.40               102        0         43 1,426.24 1,219.50   
2           3,153.83               101        0         37 1,687.25 1,387.00   
3+         13,603.94                51        0         28 2,024.31     0.00   

                         LoanAmount                                         \
           skew      std      count count_na count_zero   mean median skew   
Dependents                                                                   
0          6.61 2,630.54        334       11          0 135.89 120.00 2.88   
1          2.27 1,830.04         98        4          0 158.62 139.00 2.24   
2          4.21 2,556.27         98        3          0 150.22 133.00 1.83   
3+         5.93 6,050.79         49        2          0 190.90 130.00 2.04   

                  Loan_Amount_Term                                          \
              std            count count_na count_zero   mean median  skew   
Dependents                                                                   
0           75.85              334       11          0 348.11 360.00 -2.81   
1           95.10              101        1          0 329.35 360.00 -2.07   
2           71.28              101        0          0 340.87 360.00 -2.42   
3+         134.89               50        1          0 325.20 360.00 -1.60   

                  
             std  
Dependents        
0          57.95  
1          75.45  
2          64.91  
3+         79.57

ApplicantIncome                                             \
                       count count_na count_zero     mean   median skew   
Education                                                                 
Graduate                 480        0          0 5,857.43 4,000.00 6.02   
Not Graduate             134        0          0 3,777.28 3,357.50 3.82   

                      CoapplicantIncome                                        \
                  std             count count_na count_zero     mean   median   
Education                                                                       
Graduate     6,739.80               480        0        221 1,717.47 1,059.00   
Not Graduate 2,237.08               134        0         52 1,276.54 1,356.50   

                           LoanAmount                                         \
             skew      std      count count_na count_zero   mean median skew   
Education                                                                      
Graduate     7.03 3,230.97        465       15          0 154.06 132.00 2.46   
Not Graduate 1.11 1,310.34        127        7          0 118.41 115.00 0.69   

                   Loan_Amount_Term                                          \
               std            count count_na count_zero   mean median  skew   
Education                                                                     
Graduate     92.88              472        8          0 344.67 360.00 -2.62   
Not Graduate 39.77              128        6          0 332.16 360.00 -1.71   

                    
               std  
Education           
Graduate     61.30  
Not Graduate 77.08

ApplicantIncome                                             \
                        count count_na count_zero     mean   median skew   
Self_Employed                                                              
No                        500        0          0 5,049.75 3,705.50 7.42   
Yes                        82        0          0 7,380.82 5,809.00 2.47   

                       CoapplicantIncome                               \
                   std             count count_na count_zero     mean   
Self_Employed                                                           
No            5,682.90               500        0        213 1,580.99   
Yes           5,883.56                82        0         47 1,501.34   

                                     LoanAmount                             \
                median skew      std      count count_na count_zero   mean   
Self_Employed                                                                
No            1,293.50 6.13 2,438.16        482       18          0 141.75   
Yes               0.00 4.07 2,780.71         79        3          0 172.00   

                                 Loan_Amount_Term                             \
              median skew    std            count count_na count_zero   mean   
Self_Employed                                                                  
No            125.00 2.72  79.78              489       11          0 343.26   
Yes           150.00 2.40 108.63               80        2          0 336.30   

                                  
              median  skew   std  
Self_Employed                     
No            360.00 -2.36 64.70  
Yes           360.00 -2.47 69.40

ApplicantIncome                                             \
                         count count_na count_zero     mean   median skew   
Credit_History                                                              
0.00                        89        0          0 5,679.44 3,547.00 6.77   
1.00                       475        0          0 5,426.53 3,859.00 5.35   

                        CoapplicantIncome                               \
                    std             count count_na count_zero     mean   
Credit_History                                                           
0.00           9,301.90                89        0         41 1,542.18   
1.00           5,535.39               475        0        213 1,528.25   

                                      LoanAmount                             \
                 median skew      std      count count_na count_zero   mean   
Credit_History                                                                
0.00           1,330.00 2.19 2,023.91         85        4          0 146.72   
1.00           1,040.00 6.31 2,548.73        458       17          0 144.79   

                                 Loan_Amount_Term                             \
               median skew   std            count count_na count_zero   mean   
Credit_History                                                                 
0.00           125.00 3.08 85.16               83        6          0 341.93   
1.00           128.00 2.56 83.05              467        8          0 342.19   

                                   
               median  skew   std  
Credit_History                     
0.00           360.00 -1.33 66.74  
1.00           360.00 -2.46 64.27

ApplicantIncome                                             \
                        count count_na count_zero     mean   median skew   
Property_Area                                                              
Rural                     179        0          0 5,554.08 3,975.00 8.18   
Semiurban                 233        0          0 5,292.26 3,859.00 4.42   
Urban                     202        0          0 5,398.25 3,505.00 5.97   

                       CoapplicantIncome                               \
                   std             count count_na count_zero     mean   
Property_Area                                                           
Rural         6,782.66               179        0         72 1,645.54   
Semiurban     5,279.63               233        0        106 1,520.13   
Urban         6,392.93               202        0         95 1,716.35   

                                     LoanAmount                             \
                median skew      std      count count_na count_zero   mean   
Property_Area                                                                
Rural         1,587.00 1.11 1,785.10        173        6          0 152.26   
Semiurban       983.00 3.57 2,249.36        228        5          0 145.50   
Urban         1,007.90 7.15 4,175.10        191       11          0 142.20   

                                Loan_Amount_Term                             \
              median skew   std            count count_na count_zero   mean   
Property_Area                                                                 
Rural         133.00 2.35 80.23              175        4          0 345.53   
Semiurban     127.50 2.70 81.67              230        3          0 347.11   
Urban         120.00 2.88 94.55              195        7          0 332.80   

                                  
              median  skew   std  
Property_Area                     
Rural         360.00 -2.91 54.42  
Semiurban     360.00 -2.68 60.50  
Urban         360.00 -1.80 77.39

In [523]:
with pd.option_context("display.float_format", "{:,.2f}".format, "display.max_columns", 30):
    display(df.pivot_table(index=["Education", "Self_Employed", "Dependents"], dropna=False, aggfunc={x: ["count", "median", "mean", np.std, count_na, count_zero, pd.Series.skew] for x in NUMERICAL_FEATURES}))

ApplicantIncome                      \
                                                count count_na count_zero   
Education    Self_Employed Dependents                                       
Graduate     No            0                      231        0          0   
                           1                       59        0          0   
                           2                       59        0          0   
                           3+                      30        0          0   
             Yes           0                       29        0          0   
                           1                       17        0          0   
                           2                       14        0          0   
                           3+                       4        0          0   
Not Graduate No            0                       58        0          0   
                           1                       17        0          0   
                           2                       21        0          0   
                           3+                      12        0          0   
             Yes           0                        8        0          0   
                           1                        3        0          0   
                           2                        2        0          0   
                           3+                       3        0          0   

                                                                          \
                                           mean   median  skew       std   
Education    Self_Employed Dependents                                      
Graduate     No            0           4,566.90 3,625.00  2.72  3,343.93   
                           1           6,449.08 4,384.00  3.34  6,566.49   
                           2           5,133.32 4,283.00  1.94  2,997.80   
                           3+         11,490.50 5,208.50  3.06 17,149.10   
             Yes           0           8,282.69 6,400.00  2.60  7,668.94   
                           1           7,682.41 7,787.00  0.81  5,153.67   
                           2           6,861.93 5,873.00  1.43  4,802.81   
                           3+          7,609.75 7,517.00  0.06  2,495.24   
Not Graduate No            0           3,373.33 3,013.50  1.17  1,369.58   
                           1           3,571.94 3,399.00  0.89  1,314.42   
                           2           3,636.71 3,365.00  1.41  1,363.97   
                           3+          3,460.00 3,347.50 -0.03  1,025.17   
             Yes           0           7,825.12 5,247.50  1.21  6,292.38   
                           1           3,563.00 3,867.00 -1.59    693.88   
                           2           4,129.00 4,129.00   NaN  3,187.64   
                           3+          5,378.67 5,703.00 -0.74  1,904.33   

                                      CoapplicantIncome                      \
                                                  count count_na count_zero   
Education    Self_Employed Dependents                                         
Graduate     No            0                        231        0         96   
                           1                         59        0         26   
                           2                         59        0         26   
                           3+                        30        0         17   
             Yes           0                         29        0         21   
                           1                         17        0         10   
                           2                         14        0          5   
                           3+                         4        0          2   
Not Graduate No            0                         58        0         26   
                           1                         17        0          3   
                           2                         21        0          5   
                       

In [524]:
px.histogram(df, x="Loan_Status").show()

## 3. Data Cleaning

This step typically involves imputing missing values and treating outliers. 

### Imputing Missing Values

Missing values may not always be NaNs. For instance, the `Loan_Amount_Term` might be 0, which does not make sense.



Impute missing values for all columns. Use the values which you find most meaningful (mean, mode, median, zero.... maybe different mean values for different groups)

In [525]:
# Due to the skewness of the data the median will be used.
# It was found that at a Education, Self_Employed, Dependents groups are likely to have the highest variance, so that median will be obtained in reference to those combined subgroups.
mapLoanAmount = (
    df.groupby(by=["Education", "Self_Employed", "Dependents"])["LoanAmount"]
    .median()
    .to_dict()
)

In [526]:
def fillna_group_median(
    df: pd.DataFrame, group: list[str], fill_column: str
) -> pd.Series:
    """
    Fills na values with the the group's median for the fill_column.
    """
    print(df.shape)
    return_df = df.copy()
    group_values = df.groupby(by=group)[fill_column].median().to_dict()
    isna_mask = df[fill_column].isna()

    for key, value in group_values.items():
        groupna_mask = (df[group] == key).all(axis=1) & isna_mask
        return_df[groupna_mask] = value

    return return_df

In [527]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [528]:
filled = fillna_group_median(df, group=["Education", "Self_Employed", "Dependents"], fill_column="LoanAmount")
filled.loc[filled["LoanAmount"].isna()]

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
95,LP001326,Male,No,0,Graduate,NaN,"6,782.00",0.00,NaN,360.00,NaN,Urban,N
102,LP001350,Male,Yes,NaN,Graduate,No,"13,650.00",0.00,NaN,360.00,1.00,Urban,Y
435,LP002393,Female,NaN,NaN,Graduate,No,"10,047.00",0.00,NaN,240.00,1.00,Semiurban,Y


### Extreme values
Try a log transformation to get rid of the extreme values in `LoanAmount`. Plot the histogram before and after the transformation

In [529]:
logLoanAmount = df["LoanAmount"].apply(np.log)
px.histogram(logLoanAmount).show()

Combine both incomes as total income and take a log transformation of the same.

In [530]:
totalIncome = (df["ApplicantIncome"] + df["CoapplicantIncome"]).apply(np.log)
px.histogram(totalIncome).show()

## 4. Building a Predictive Model

### Preprocessing
#### Categorical features:
1. Fillna's
    1. Dependents = mode
    2. Self_Employed = mode
    3. Credit_History = mode
2. OneHotEncoder

#### Numerical features:
1. Fillna's with median
    1. LoanAmount = group median
    2. LoanTerm = median
2. Create totalIncome column (ApplicantIncome + CoapplicantIncome)
3. Logtransform totalIncome
4. StandardScaler all columns

### Main Pipeline
1. FeatureUnion categorical features and numerical features.
2. Classifiers:
    1. RandomForestClassifier
    2. LogisticRegression
3. GridsearchCV for cross validation and hyperparameter tuning

In [531]:
import numpy.typing as npt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, FunctionTransformer, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV

### Test / Train split

In [532]:
y = df["Loan_Status"].replace({"Y": 1, "N": 0})
X = df.drop(columns=["Loan_Status"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Custom transformers

In [533]:
# class CreateTotalIncome():
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X: npt.NDArray):
#         total_income = (X[:, 0] + X[:, 1]).reshape(-1, 1)
#         X = np.concatenate((X, total_income), axis=1)
#         return total_income.reshape(-1, 1)

from custom_transformers import CreateTotalIncome

### Pipelines

In [534]:
fillna_pipe = Pipeline(
    [
        (
            "fillna",
            ColumnTransformer(
                [
                    (
                        "categorical",
                        SimpleImputer(strategy="most_frequent"),
                        CATEGORICAL_FEATURES,
                    ),
                    ("numerical", SimpleImputer(strategy="mean"), NUMERICAL_FEATURES),
                ],
                remainder="drop",
            ),
        )
    ]
)

categorical_pipe = Pipeline(
    [
        (
            "fillna",
            ColumnTransformer(
                [
                    (
                        "categorical",
                        SimpleImputer(strategy="most_frequent"),
                        CATEGORICAL_FEATURES,
                    ),
                ],
                remainder="drop",
            ),
        ),
        ("one-hot-encoding", OneHotEncoder(drop="if_binary", sparse=False)),
    ]
)

totalIncome_pipe = Pipeline(
    [
        (
            "fillna",
            ColumnTransformer(
                [
                    (
                        "income",
                        SimpleImputer(strategy="mean"),
                        ["ApplicantIncome", "CoapplicantIncome"],
                    ),
                ],
                remainder="drop",
            ),
        ),
        ("create_feature", CreateTotalIncome()),
        ("log_transform", FunctionTransformer(np.log)),
        ("scale", StandardScaler()),
    ]
)

other_numerical_columns = Pipeline(
    [
        (
            "fillna",
            ColumnTransformer(
                [
                    (
                        "other_numerical",
                        SimpleImputer(strategy="median"),
                        ["LoanAmount", "Loan_Amount_Term"],
                    ),
                ],
                remainder="drop",
            ),
        ),
        ("scale", StandardScaler()),
    ]
)

rfc = RandomForestClassifier(max_depth=5, n_jobs=-1)
lr = LogisticRegression(n_jobs=-1)

classifier_pipe = Pipeline(
    [
        ("join_features", FeatureUnion([("categorical", categorical_pipe), ("other_numerical", other_numerical_columns), ("total_income", totalIncome_pipe)])),
        ("classifier", rfc),
    ]
)

In [535]:
classifier_pipe.fit(X_train, y_train)

Pipeline(steps=[('join_features',
                 FeatureUnion(transformer_list=[('categorical',
                                                 Pipeline(steps=[('fillna',
                                                                  ColumnTransformer(transformers=[('categorical',
                                                                                                   SimpleImputer(strategy='most_frequent'),
                                                                                                   ['Dependents',
                                                                                                    'Education',
                                                                                                    'Self_Employed',
                                                                                                    'Credit_History',
                                                                                                    'Property_Area'])])),
                                                                 ('one-hot-encoding',
                                                                  OneHotEncoder(drop='if_binary',
                                                                                sparse=False))])),
                                                ('other_n...
                                                 Pipeline(steps=[('fillna',
                                                                  ColumnTransformer(transformers=[('income',
                                                                                                   SimpleImputer(),
                                                                                                   ['ApplicantIncome',
                                                                                                    'CoapplicantIncome'])])),
                                                                 ('create_feature',
                                                                  <custom_transformers.CreateTotalIncome object at 0x7f329d8518a0>),
                                                                 ('log_transform',
                                                                  FunctionTransformer(func=<ufunc 'log'>)),
                                                                 ('scale',
                                                                  StandardScaler())]))])),
                ('classifier', RandomForestClassifier(max_depth=5, n_jobs=-1))])

In [536]:
classifier_pipe.score(X_test, y_test)

0.7783783783783784

Try paramater grid search to improve the results

In [537]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

params = {
    "classifier": [
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        GaussianProcessClassifier(1.0 * RBF(1.0)),
        RandomForestClassifier(max_depth=5),
        AdaBoostClassifier(),
        GaussianNB(),
    ],
}

grid = GridSearchCV(classifier_pipe, param_grid=params, n_jobs=-1, scoring="roc_auc", cv=10)
grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('join_features',
                                        FeatureUnion(transformer_list=[('categorical',
                                                                        Pipeline(steps=[('fillna',
                                                                                         ColumnTransformer(transformers=[('categorical',
                                                                                                                          SimpleImputer(strategy='most_frequent'),
                                                                                                                          ['Dependents',
                                                                                                                           'Education',
                                                                                                                           'Self_Employed',
                                                                                                                           'Credit_History',
                                                                                                                           'Property_Area'])])),
                                                                                        ('one-hot-encoding',
                                                                                         OneHotEncoder(drop='if_binary...
                                                                                         StandardScaler())]))])),
                                       ('classifier',
                                        RandomForestClassifier(max_depth=5,
                                                               n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'classifier': [KNeighborsClassifier(n_neighbors=3),
                                        SVC(C=0.025, kernel='linear'),
                                        SVC(C=1, gamma=2),
                                        GaussianProcessClassifier(kernel=1**2 * RBF(length_scale=1)),
                                        RandomForestClassifier(max_depth=5),
                                        AdaBoostClassifier(), GaussianNB()]},
             scoring='roc_auc')

In [538]:
print(grid.best_score_)
print(grid.best_params_)

0.7606134546457126
{'classifier': GaussianNB()}


## 5. Using Pipeline
If you didn't use pipelines before, transform your data prep, feat. engineering and modeling steps into Pipeline. It will be helpful for deployment.

The goal here is to create the pipeline that will take one row of our dataset and predict the probability of being granted a loan.

`pipeline.predict(x)`

In [539]:
sample = X_test.sample()
print(grid.predict_proba(sample))
print(grid.predict(sample))

[[1.00000000e+00 1.63207034e-20]]
[0]


## 6. Deploy your model to cloud and test it with PostMan, BASH or Python

In [540]:
import joblib

In [541]:
with open("../src/model.joblib", "wb") as f:
    joblib.dump(grid, f)    